# PEFTのアダプタを読み込む

Based on <https://huggingface.co/docs/transformers/ja/peft>

PEFTアダプターを読み込む方法は2つある。

1. adapter_config.jsonのあるモデルをロードする。モデルとアダプターがセットで読み込まれる
2. モデルを読み込んでから、アダプタを追加で読み込む

読み込み時に量子化できる。学習用に新規のアダプタを作成できる。

In [ ]:
# モデルとアダプタをセットで読み込む例

from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "ybelkada/opt-350m-lora"
model = AutoModelForCausalLM.from_pretrained(peft_model_id)

ここで読み込んでいるアダプター: <https://huggingface.co/ybelkada/opt-350m-lora/tree/main>

<https://huggingface.co/facebook/opt-350m> に対するアダプタだが、何を学習済みなのかは不明。


In [ ]:
# モデルを読み込んでから、アダプタを追加で読み込む

from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "facebook/opt-350m"
peft_model_id = "ybelkada/opt-350m-lora"

model = AutoModelForCausalLM.from_pretrained(model_id)
model.load_adapter(peft_model_id)

In [ ]:
# モデルとアダプタをセットで、量子化して読み込む(=QloRA)

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

peft_model_id = "ybelkada/opt-350m-lora"
model = AutoModelForCausalLM.from_pretrained(peft_model_id, quantization_config=BitsAndBytesConfig(load_in_8bit=True))

In [ ]:
# アダプタを新規に作成してアタッチする

from transformers import AutoModelForCausalLM, OPTForCausalLM, AutoTokenizer
from peft import LoraConfig

model_id = "facebook/opt-350m"
model = AutoModelForCausalLM.from_pretrained(model_id)

lora_config = LoraConfig(
    target_modules=["q_proj", "k_proj"],
    init_lora_weights=False
)

model.add_adapter(lora_config, adapter_name="adapter_1")

以下はアダプタを複数作って使用時に選択する例

In [ ]:
# attach new adapter with same config
model.add_adapter(lora_config, adapter_name="adapter_2")

In [ ]:
## use adapter_1
# model.set_adapter("adapter_1")
# output = model.generate(**inputs)
# print(tokenizer.decode(output_disabled[0], skip_special_tokens=True))

## use adapter_2
# model.set_adapter("adapter_2")
# output_enabled = model.generate(**inputs)
# print(tokenizer.decode(output_enabled[0], skip_special_tokens=True))

## PEFTアダプターを学習する

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
model.add_adapter(peft_config)

あとはTrainerで学習するだけ

In [ ]:
trainer = Trainer(model=model, ...)
trainer.train()

こうなると LoraConfigの内容を理解するほうが先決そう。

上の例では中間層の数 r=64 、biasなし、CAUSAL_LMに対するアダプテーションだということはわかる。
alphaとdropoutがわかってない。

[ybelkada/opt-350m-lora](https://huggingface.co/ybelkada/opt-350m-lora/blob/main/adapter_config.json) の alpha と dropout は 32 と 0.05 で r=16

[LoraConfigのドキュメント](https://huggingface.co/docs/peft/package_reference/lora#peft.LoraConfig)

全部で19個のパラメータ、以下は抜粋:

* r - attention の時限
* alpha - LoRA scaling ?
* dropout - LoRA層のドロップアウト確率 ?
* bias - バイアスのタイプ: none, all, lora_only

[C3TR-Adapterのconfig](https://huggingface.co/webbigdata/C3TR-Adapter/blob/main/adapter_config.json)

* r=128
* alpha=64
* dropout=0
* target_modules: gate_proj, v_proj, k_proj, o_proj, up_proj, q_proj, down_proj

gemma2に対してたったr=128で翻訳できるようになってるの、ちょっと驚き